# CDAT Migration Regression Testing Notebook (`.nc` files)

This notebook is used to perform regression testing between the development and
production versions of a diagnostic set.

## How it works

It compares the relative differences (%) between ref and test variables between
the dev and `main` branches.

## How to use

PREREQUISITE: The diagnostic set's netCDF stored in `.json` files in two directories
(dev and `main` branches).

1. Make a copy of this notebook under `auxiliary_tools/cdat_regression_testing/<DIR_NAME>`.
2. Run `mamba create -n cdat_regression_test -y -c conda-forge "python<3.12" xarray netcdf4 dask pandas matplotlib-base ipykernel`
3. Run `mamba activate cdat_regression_test`
4. Update `SET_DIR` and `SET_NAME` in the copy of your notebook.
5. Run all cells IN ORDER.
6. Review results for any outstanding differences (>=1e-5 relative tolerance).
   - Debug these differences (e.g., bug in metrics functions, incorrect variable references, etc.)


## Setup Code


In [15]:
import glob

import numpy as np
import xarray as xr
from e3sm_diags.derivations.derivations import DERIVED_VARIABLES

SET_NAME = "enso_diags"
SET_DIR = "663-enso_diags"

DEV_PATH = f"/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/{SET_DIR}/{SET_NAME}/**"
DEV_GLOB = sorted(glob.glob(DEV_PATH + "/*.nc"))
DEV_NUM_FILES = len(DEV_GLOB)

MAIN_PATH = f"/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main/{SET_NAME}/**"
MAIN_GLOB = sorted(glob.glob(MAIN_PATH + "/*.nc"))
MAIN_NUM_FILES = len(MAIN_GLOB)

In [16]:
def _check_if_files_found():
    if DEV_NUM_FILES == 0 or MAIN_NUM_FILES == 0:
        raise IOError(
            "No files found at DEV_PATH and/or MAIN_PATH. "
            f"Please check {DEV_PATH} and {MAIN_PATH}."
        )


def _check_if_matching_filecount():
    if DEV_NUM_FILES != MAIN_NUM_FILES:
        raise IOError(
            "Number of files do not match at DEV_PATH and MAIN_PATH "
            f"({DEV_NUM_FILES} vs. {MAIN_NUM_FILES})."
        )

    print(f"Matching file count ({DEV_NUM_FILES} and {MAIN_NUM_FILES}).")


def _check_if_missing_files():
    missing_count = 0

    for fp_main in MAIN_GLOB:
        fp_dev = fp_main.replace(SET_DIR, "main")

        if fp_dev not in MAIN_GLOB:
            print(f"No production file found to compare with {fp_dev}!")
            missing_count += 1

    for fp_dev in DEV_GLOB:
        fp_main = fp_main.replace("main", SET_DIR)

        if fp_main not in DEV_GLOB:
            print(f"No development file found to compare with {fp_main}!")
            missing_count += 1

    print(f"Number of files missing: {missing_count}")

In [102]:
def _get_relative_diffs() -> int:
    # We are mainly focusing on relative tolerance here (in percentage terms).
    atol = 0
    rtol = 1e-5

    mismatches = []

    for fp_main in MAIN_GLOB:
        if "test.nc" in fp_main or "ref.nc" in fp_main:
            fp_dev = fp_main.replace("main", SET_DIR)

            print("Comparing:")
            print(f"    * {fp_dev}")
            print(f"    * {fp_main}")

            ds1 = xr.open_dataset(fp_dev)
            ds2 = xr.open_dataset(fp_main)

            var_key = fp_main.split("-")[-3].upper()
            var_key_cdat = f"{var_key}-regression-over-nino"
            print(f"    * var_key: {var_key}")

            dev_data = ds1[var_key].values
            main_data = ds2[var_key_cdat].values

            if dev_data is None or main_data is None:
                print("    * Could not find variable key in the dataset(s)")
                continue

            try:
                np.testing.assert_allclose(
                    dev_data,
                    main_data,
                    atol=atol,
                    rtol=rtol,
                )
            except (KeyError, AssertionError) as e:
                print(f"    {e}")
                mismatches.append((fp_dev, fp_main))
            else:
                print(f"    * All close and within relative tolerance ({rtol})")

    return mismatches

## 1. Check for matching and equal number of files


In [103]:
_check_if_files_found()

In [104]:
DEV_GLOB

['/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/663-enso_diags/enso_diags/LHFLX-response/regression-coefficient-lhflx-over-nino34_diff.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/663-enso_diags/enso_diags/LHFLX-response/regression-coefficient-lhflx-over-nino34_ref.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/663-enso_diags/enso_diags/LHFLX-response/regression-coefficient-lhflx-over-nino34_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/663-enso_diags/enso_diags/NET_FLUX_SRF-response/regression-coefficient-net_flux_srf-over-nino34_diff.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/663-enso_diags/enso_diags/NET_FLUX_SRF-response/regression-coefficient-net_flux_srf-over-nino34_ref.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/663-enso_diags/enso_diags/NET_FLUX_SRF-response/regression-coefficient-net_flux_srf-over-nino34_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/663-enso_diags/enso_diags/PRECT-response/regression-coefficient-p

In [105]:
MAIN_GLOB

['/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main/enso_diags/LHFLX-response/regression-coefficient-lhflx-over-nino34_diff.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main/enso_diags/LHFLX-response/regression-coefficient-lhflx-over-nino34_ref.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main/enso_diags/LHFLX-response/regression-coefficient-lhflx-over-nino34_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main/enso_diags/NET_FLUX_SRF-response/regression-coefficient-net_flux_srf-over-nino34_diff.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main/enso_diags/NET_FLUX_SRF-response/regression-coefficient-net_flux_srf-over-nino34_ref.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main/enso_diags/NET_FLUX_SRF-response/regression-coefficient-net_flux_srf-over-nino34_test.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main/enso_diags/PRECT-response/regression-coefficient-prect-over-nino34_diff.nc',
 '/global/cfs/cdirs/e3sm/www/cdat-migration

In [106]:
_check_if_missing_files()

Number of files missing: 0


In [107]:
_check_if_matching_filecount()

Matching file count (18 and 18).


## 2 Compare the netCDF files between branches

- Compare "ref" and "test" files
- "diff" files are ignored because getting relative diffs for these does not make sense (relative diff will be above tolerance)


In [108]:
mismatches = _get_relative_diffs()

Comparing:
    * /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/663-enso_diags/enso_diags/LHFLX-response/regression-coefficient-lhflx-over-nino34_ref.nc
    * /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main/enso_diags/LHFLX-response/regression-coefficient-lhflx-over-nino34_ref.nc
    * var_key: LHFLX
    
Not equal to tolerance rtol=1e-05, atol=0

(shapes (53, 480), (55, 480) mismatch)
 x: array([[ 0.372228,  0.4754  ,  0.693551, ...,  0.04266 ,  0.275301,
         0.294088],
       [ 0.576394,  0.650426,  0.902666, ...,  0.179812,  0.45417 ,...
 y: array([[ 0.345141,  0.473852,  0.6088  , ..., -0.014503,  0.138434,
         0.257044],
       [ 0.372228,  0.4754  ,  0.693551, ...,  0.04266 ,  0.275301,...
Comparing:
    * /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/663-enso_diags/enso_diags/LHFLX-response/regression-coefficient-lhflx-over-nino34_test.nc
    * /global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main/enso_diags/LHFLX-response/regression-coefficient-lhflx-over-nino34_te

In [109]:
mismatches

[('/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/663-enso_diags/enso_diags/LHFLX-response/regression-coefficient-lhflx-over-nino34_ref.nc',
  '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main/enso_diags/LHFLX-response/regression-coefficient-lhflx-over-nino34_ref.nc'),
 ('/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/663-enso_diags/enso_diags/NET_FLUX_SRF-response/regression-coefficient-net_flux_srf-over-nino34_ref.nc',
  '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main/enso_diags/NET_FLUX_SRF-response/regression-coefficient-net_flux_srf-over-nino34_ref.nc'),
 ('/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/663-enso_diags/enso_diags/SHFLX-response/regression-coefficient-shflx-over-nino34_ref.nc',
  '/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/main/enso_diags/SHFLX-response/regression-coefficient-shflx-over-nino34_ref.nc'),
 ('/global/cfs/cdirs/e3sm/www/cdat-migration-fy24/663-enso_diags/enso_diags/TAUX-response/regression-coefficient-taux-over-nino34_ref.nc',
  '/global/cfs